# U-Net Segmentation - Camus Dataset

*Made by **Hang Jung Ling** and **Olivier Bernard** from the INSA Lyon, France.*

This notebook shows how to train, test and evaluate a U-Net to segment different cardiac structures on [CAMUS dataset](https://humanheart-project.creatis.insa-lyon.fr/database/#collection/6373703d73e9f0047faa1bc8).

CAMUS is one of the largest public echocardiogaphic datasets, with 500 patients and each patient has 4 echocardiographic images: end-diastolic (ED) and end-systolic (ES) frames acquired in both apical two chamber and apical four chamber views. Each image is annotated by an expert and contains 3 classes + background:</br>
&emsp;1) Left ventricle</br>
&emsp;2) Myocardium</br>
&emsp;3) Left atrium</br>

Summary :</br>
&emsp;I.   [Install dependencies](#install)</br>
&emsp;II.  [Dataset](#dataset)</br>
&emsp;II.  [Train](#train)</br>
&emsp;III. [Visualize](#visualize)</br>
&emsp;V.   [Evaluate](#evaluate)</br> 


# I. Install dependencies <a class="anchor" id="install"></a>

Kindly ignore this step if you have installed your own environment using `environment.yaml`. If not, please execute the following cells to install the dependencies.

In [ ]:
%%capture project_path_setup

import sys

if "../" in sys.path:
    print(sys.path)
else:
    sys.path.append("../")
    print(sys.path)

In [ ]:
%%capture packages_install

# Make sure the repo's package and its dependencies are installed
%pip install -e ../.

# II. Dataset <a class="anchor" id="dataset"></a>

Once the environment is successfully setup, download the CAMUS dataset by executing the following cell. The dataset will be downloaded to the `data/` folder.

In [ ]:
from pathlib import Path

from tqdm.auto import tqdm

# Make sure the data is downloaded and extracted where it should be
if not Path("../data/camus_64").is_dir():
    import zipfile
    from io import BytesIO
    from urllib.request import urlopen

    zipurl = "https://www.creatis.insa-lyon.fr/~bernard/camus/camus_64.zip"
    with urlopen(zipurl) as zipresp:
        with zipfile.ZipFile(BytesIO(zipresp.read())) as zfile:
            for member in tqdm(
                zfile.infolist(), desc="Downloading and extracting data", position=0, leave=True
            ):
                try:
                    zfile.extract(member, "../data/")
                except zipfile.error as e:
                    pass

Now, let's split these data into training, validation and testing sets. We will use 80% of the data for training, 10% for validation and 10% for testing. The split is done by patient ID, so that the same patient will not appear in different sets.

In [ ]:
from sklearn.model_selection import train_test_split

from src.utils.file_and_folder_operations import subdirs

# Specify the data directory
data_dir = Path("../data/camus_64").resolve()

# List all the patients id
keys = subdirs(data_dir, prefix="patient", join=False)

# Split the patients into 80/10/10 train/val/test sets
train_keys, val_and_test_keys = train_test_split(keys, train_size=0.8, random_state=12345)
val_keys, test_keys = train_test_split(val_and_test_keys, test_size=0.5, random_state=12345)

train_keys = sorted(train_keys)
val_keys = sorted(val_keys)
test_keys = sorted(test_keys)

# Create train, val and test datalist
viws_instants = ["2CH_ED", "2CH_ES", "4CH_ED", "4CH_ES"]
train_datalist = [
    {
        "image": str(data_dir / key / f"{key}_{view}.nii.gz"),
        "label": str(data_dir / key / f"{key}_{view}_gt.nii.gz"),
    }
    for key in train_keys
    for view in viws_instants
]

val_datalist = [
    {
        "image": str(data_dir / key / f"{key}_{view}.nii.gz"),
        "label": str(data_dir / key / f"{key}_{view}_gt.nii.gz"),
    }
    for key in val_keys
    for view in viws_instants
]

test_datalist = [
    {
        "image": str(data_dir / key / f"{key}_{view}.nii.gz"),
        "label": str(data_dir / key / f"{key}_{view}_gt.nii.gz"),
    }
    for key in test_keys
    for view in viws_instants
]

print("Example of train keys: ", train_datalist[:2])
print("Example of validation keys: ", val_datalist[:2])
print("Example of test keys: ", test_datalist[:2])

Once the data is split, we will create a `Dataset` object for each set. This object will be used to load the data during training and testing.

In [ ]:
import numpy as np
from monai.data import CacheDataset
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    LoadImaged,
    NormalizeIntensityd,
    RandAdjustContrastd,
    RandFlipd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    RandRotated,
    RandScaleIntensityd,
    RandZoomd,
)

# Transforms to load data
load_transforms = [
    LoadImaged(keys=["image", "label"], image_only=True),  # Load image and label
    EnsureChannelFirstd(
        keys=["image", "label"]
    ),  # Make sure the first dimension is the channel dimension
    NormalizeIntensityd(keys=["image"]),  # Normalize the intensity of the image
]

# Transforms to augment data
range_x = [-15.0 / 180 * np.pi, 15.0 / 180 * np.pi]
data_augmentation_transforms = [
    RandRotated(
        keys=["image", "label"],
        range_x=range_x,
        range_y=0,
        range_z=0,
        mode=["bicubic", "nearest"],
        padding_mode="constant",
        prob=0.2,
    ),
    RandZoomd(
        keys=["image", "label"],
        min_zoom=0.7,
        max_zoom=1.4,
        mode=["bicubic", "nearest"],
        padding_mode="constant",
        align_corners=(True, None),
        prob=0.2,
    ),
    RandGaussianNoised(keys=["image"], std=0.01, prob=0.15),
    RandGaussianSmoothd(
        keys=["image"],
        sigma_x=(0.5, 1.15),
        sigma_y=(0.5, 1.15),
        prob=0.15,
    ),
    RandScaleIntensityd(keys=["image"], factors=0.3, prob=0.15),
    RandAdjustContrastd(keys=["image"], gamma=(0.7, 1.5), prob=0.3),
    RandFlipd(keys=["image", "label"], spatial_axis=[0], prob=0.5),
]

# Define transforms for training, validation and testing
train_transforms = Compose(load_transforms + data_augmentation_transforms)
val_transforms = Compose(load_transforms)
test_transforms = Compose(load_transforms)

train_ds = CacheDataset(data=train_datalist, transform=train_transforms, cache_rate=1.0)
val_ds = CacheDataset(data=val_datalist, transform=val_transforms, cache_rate=1.0)
test_ds = CacheDataset(data=test_datalist, transform=test_transforms, cache_rate=1.0)

Now, let's visualize some images from the training set. The images are displayed with their corresponding ground truth segmentation masks.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

from src.utils.visualizations import imagesc

# Get a random image with label from each dataset
train_idx = np.random.randint(len(train_ds))
val_idx = np.random.randint(len(val_ds))
test_idx = np.random.randint(len(test_ds))

print("train_idx: ", train_idx)
print("val_idx: ", val_idx)
print("test_idx: ", test_idx)

# Visualize a random image with label from each dataset
colors = ["black", "red", "green", "blue"]
cmap = ListedColormap(colors)

figure = plt.figure(figsize=(8, 8))
train_sample = train_ds[train_idx]
image = train_sample["image"].detach().cpu().numpy()[0].transpose(1, 0)
label = train_sample["label"].detach().cpu().numpy()[0].transpose(1, 0)
ax = figure.add_subplot(3, 2, 1)
imagesc(ax, image, title="Training image", show_colorbar=False)
ax = figure.add_subplot(3, 2, 2)
plt.imshow(label, cmap=cmap, interpolation="nearest")
plt.title("Training label")
ax.axis("off")

val_sample = val_ds[val_idx]
image = val_sample["image"].detach().cpu().numpy()[0].transpose(1, 0)
label = val_sample["label"].detach().cpu().numpy()[0].transpose(1, 0)
ax = figure.add_subplot(3, 2, 3)
imagesc(ax, image, title="Validation image", show_colorbar=False)
ax = figure.add_subplot(3, 2, 4)
plt.imshow(label, cmap=cmap, interpolation="nearest")
plt.title("Validation label")
ax.axis("off")

test_sample = test_ds[test_idx]
image = test_sample["image"].detach().cpu().numpy()[0].transpose(1, 0)
label = test_sample["label"].detach().cpu().numpy()[0].transpose(1, 0)
ax = figure.add_subplot(3, 2, 5)
imagesc(ax, image, title="Test image", show_colorbar=False)
ax = figure.add_subplot(3, 2, 6)
plt.imshow(label, cmap=cmap, interpolation="nearest")
plt.title("Test label")
ax.axis("off")
figure.tight_layout()
plt.show()

# III. Train <a class="anchor" id="train"></a>
Let's move on to train a U-Net to segment the left ventricle, myocardium and left atrium. We will use the training and validation sets created in the previous section.

### Definition of U-Net architecture

In [ ]:
from torchinfo import summary

from src.models.unet import UNet

input_channels = 1  # This is the number of input channels in the image
input_shape = (input_channels, 64, 64)  # This is the shape of the input image to the network
num_classes = 4  # This is the number of output classes
output_shape = (num_classes, 64, 64)  # This is the shape of the output mask
init_channels = 32  # This is the number of channels in the first layer of the network

unet = UNet(input_shape=input_shape, output_shape=output_shape, init_channels=init_channels)

# Print the summary of the network
summary_kwargs = dict(
    col_names=["input_size", "output_size", "kernel_size", "num_params"], depth=3, verbose=0
)
summary(unet, (1, *input_shape), device="cpu", **summary_kwargs)

### Definition of optimizer, loss function, and metrics

In [ ]:
from functools import partial

import torch
from monai.losses import DiceCELoss
from monai.metrics import DiceMetric

# Soft dice and CE loss function
loss_function = DiceCELoss(
    include_background=False,
    to_onehot_y=True,
    batch=True,
    smooth_nr=0.00001,
    smooth_dr=0.00001,
    lambda_dice=0.5,
    lambda_ce=0.5,
)

# Adam optimizer
optimizer = partial(torch.optim.Adam, params=unet.parameters())

# Hard dice metric
metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)

### Definition of training function

In [ ]:
import os
import time
from typing import Union

from monai.data import DataLoader, decollate_batch
from monai.transforms import Activations, AsDiscrete
from torch import nn
from torch.utils.data import Dataset


def train_process(
    train_ds: Dataset,
    val_ds: Dataset,
    model: nn.Module,
    loss_function: nn.Module,
    optimizer: torch.optim.Optimizer,
    metric: nn.Module,
    num_classes: int,
    batch_size: int = 2,
    lr: float = 0.001,
    max_epochs: int = 30,
    log_dir: Union[Path, str] = Path("../logs/camus_segmentation"),
    val_interval=1,
):
    # Create train and validation dataloaders
    train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=os.cpu_count() - 1,
        pin_memory=torch.cuda.is_available(),
    )
    val_dataloader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=os.cpu_count() - 1,
        pin_memory=torch.cuda.is_available(),
    )

    # Determine the device to run the model on
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"GPU detected, training on: {device}!\n")
    else:
        device = torch.device("cpu")
        print("GPU not detected, training on CPU!\n")

    # Move the model to the device
    model = model.to(device)

    # Finalize the creation of the optimizer object with the learning rate
    optimizer = optimizer(lr=lr)

    # Define transforms for post processing predictions and labels
    post_pred = Compose(
        [Activations(softmax=True), AsDiscrete(argmax=True, to_onehot=num_classes)]
    )
    post_label = Compose([AsDiscrete(to_onehot=num_classes)])

    # Convert log directory to Path object if needed
    if not isinstance(log_dir, Path):
        log_dir = Path(log_dir)
    # Create the log directory if it does not exist
    log_dir.mkdir(parents=True, exist_ok=True)

    # Define some variables to keep track of the best metric values, epoch time and losses
    best_metric = -1
    best_metric_epoch = -1
    epoch_train_loss_values = []
    epoch_val_loss_values = []
    metric_values = []
    epoch_times = []
    epoch_val = []
    total_start = time.time()

    fit_pbar = tqdm(range(max_epochs), desc="Training", unit="epoch", position=0, leave=True)
    pbar_metrics = {"train/loss": None, "val/loss": None, "val/metric": None}
    fit_pbar.set_postfix(pbar_metrics)

    for epoch in fit_pbar:
        epoch_start = time.time()
        model.train()
        epoch_loss = 0
        epoch_val_loss = 0
        step = 0
        for batch_data in train_dataloader:
            step += 1
            inputs, labels = (
                batch_data["image"].to(device),
                batch_data["label"].to(device),
            )
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            pbar_metrics["loss"] = loss.item()
            pbar_metrics["train_batch"] = f"{step}/{len(train_dataloader)}"
            fit_pbar.set_postfix(pbar_metrics)

        epoch_loss /= step
        epoch_train_loss_values.append(epoch_loss)
        pbar_metrics["train/loss"] = epoch_loss
        pbar_metrics.pop("loss")
        pbar_metrics.pop("train_batch")
        fit_pbar.set_postfix(pbar_metrics)

        if (epoch + 1) % val_interval == 0:
            step = 0
            model.eval()
            with torch.no_grad():
                for val_data in val_dataloader:
                    step += 1
                    val_inputs, val_labels = (
                        val_data["image"].to(device),
                        val_data["label"].to(device),
                    )
                    val_outputs = model(val_inputs)
                    val_loss = loss_function(val_outputs, val_labels)
                    epoch_val_loss += val_loss.item()
                    val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                    val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                    metric(y_pred=val_outputs, y=val_labels)
                    pbar_metrics["loss"] = val_loss.item()
                    pbar_metrics["val_batch"] = f"{step}/{len(val_dataloader)}"
                    fit_pbar.set_postfix(pbar_metrics)

                epoch_val_loss /= step
                epoch_val_loss_values.append(epoch_val_loss)
                epoch_val.append(epoch + 1)
                val_metric = metric.aggregate().item()
                metric.reset()
                metric_values.append(val_metric)
                pbar_metrics.pop("loss")
                pbar_metrics.pop("val_batch")
                pbar_metrics["val/loss"] = epoch_val_loss
                pbar_metrics["val/metric"] = val_metric
                fit_pbar.set_postfix(pbar_metrics)

                if val_metric > best_metric:
                    best_metric = val_metric
                    best_metric_epoch = epoch + 1
                    torch.save(
                        model.state_dict(),
                        str(log_dir / "best_metric_model.pth"),
                    )
                    # print("saved new best metric model at epoch", best_metric_epoch)
        epoch_times.append(time.time() - epoch_start)

    print(
        f"train completed, best_metric: {best_metric:.4f}"
        f" at epoch: {best_metric_epoch}"
        f" total time: {(time.time() - total_start):.4f}"
    )
    return (
        max_epochs,
        time.time() - total_start,
        epoch_train_loss_values,
        epoch_val_loss_values,
        epoch_val,
        metric_values,
        epoch_times,
    )

### Definition of training hyperparameters
In this section, we will define the hyperparameters for training, such as the number of epochs, the learning rate, the batch size, etc.

In [ ]:
batch_size = 2
lr = 0.001
max_epochs = 30

# Train the model
(
    max_epochs,
    total_time,
    epoch_train_loss_values,
    epoch_val_loss_values,
    epoch_val,
    metric_values,
    epoch_times,
) = train_process(
    train_ds=train_ds,
    val_ds=val_ds,
    model=unet,
    loss_function=loss_function,
    optimizer=optimizer,
    metric=metric,
    num_classes=num_classes,
    batch_size=batch_size,
    lr=lr,
    max_epochs=max_epochs,
    log_dir="../logs/camus_segmentation",
)